In [1]:
import pandas as pd
import numpy as np 
import matplotlib as mpl
import seaborn as sns

In [2]:
FEATURE = "Infant_Mortality_Rate"
df = pd.read_csv("..\..\datasets\Original\%s.csv" % FEATURE)
df

,Series Name,Series Code,Country Name,Country Code,2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,"Mortality rate, under-5 (per 1,000)",SH.DYN.MORT,Afghanistan,AFG,67.5,64.8,62.3,60.1,58
1,"Mortality rate, under-5 (per 1,000)",SH.DYN.MORT,Albania,ALB,9.4,9.4,9.5,9.7,9.8
2,"Mortality rate, under-5 (per 1,000)",SH.DYN.MORT,Algeria,DZA,24.8,24.3,23.8,23.3,22.7
3,"Mortality rate, under-5 (per 1,000)",SH.DYN.MORT,American Samoa,ASM,..,..,..,..,..
4,"Mortality rate, under-5 (per 1,000)",SH.DYN.MORT,Andorra,AND,3.2,3,2.8,2.7,2.5
...,...,...,...,...,...,...,...,...,...
212,"Mortality rate, under-5 (per 1,000)",SH.DYN.MORT,Virgin Islands (U.S.),VIR,..,..,..,..,..
213,"Mortality rate, under-5 (per 1,000)",SH.DYN.MORT,West Bank and Gaza,PSE,18.6,18.1,17.5,17,16.5
214,"Mortality rate, under-5 (per 1,000)",SH.DYN.MORT,"Yemen, Rep.",YEM,60.3,60.2,61.6,61.5,59.6
215,"Mortality rate, under-5 (per 1,000)",SH.DYN.MORT,Zambia,ZMB,65.3,64.6,65.4,64.1,61.4


## Skeleton dataframe

In [3]:
columns = ["Country Code"]
df.drop_duplicates(subset=columns, keep="first")
df_new = df[columns[0]].unique()
df_new = pd.DataFrame(df_new, columns=columns)
df.rename(columns={"Country Code": "ISO3_Code"}, inplace=True)
df_new.rename(columns={"Country Code": "ISO3_Code"}, inplace=True)
df_new.shape

(217, 1)

In [4]:
df_code = pd.read_csv("../../datasets/Original/AreaCode_ISO3Code_conversion.csv")
df_code = df_code[["Country", "ISO3 Code", "M49 Code"]]
df_new = df_new.join(df_code.set_index("ISO3 Code"), on="ISO3_Code", validate="1:1")
df_new

,ISO3_Code,Country,M49 Code
0,AFG,Afghanistan,4.0
1,ALB,Albania,8.0
2,DZA,Algeria,12.0
3,ASM,American Samoa,16.0
4,AND,Andorra,20.0
...,...,...,...
212,VIR,United States Virgin Islands,850.0
213,PSE,Palestine,275.0
214,YEM,Yemen,887.0
215,ZMB,Zambia,894.0


In [5]:
df_new.rename(columns={"ISO3 Code": "ISO3_Code", "Country": "Country_Name", "M49 Code": "M49_Code"}, inplace=True)
df_new.head()

,ISO3_Code,Country_Name,M49_Code
0,AFG,Afghanistan,4.0
1,ALB,Albania,8.0
2,DZA,Algeria,12.0
3,ASM,American Samoa,16.0
4,AND,Andorra,20.0


In [6]:
df_new[df_new["Country_Name"].isnull()]

,ISO3_Code,Country_Name,M49_Code
39,CHI,NaN,NaN
105,XKX,NaN,NaN


In [7]:
# drop the null rows cause it's the legend
df_new.dropna(inplace=True)
df_new

,ISO3_Code,Country_Name,M49_Code
0,AFG,Afghanistan,4.0
1,ALB,Albania,8.0
2,DZA,Algeria,12.0
3,ASM,American Samoa,16.0
4,AND,Andorra,20.0
...,...,...,...
212,VIR,United States Virgin Islands,850.0
213,PSE,Palestine,275.0
214,YEM,Yemen,887.0
215,ZMB,Zambia,894.0


## Unemployment
Unemployment (% of labor force)

In [8]:
df_tmp = pd.concat([df.loc[:, ["ISO3_Code"]], df.iloc[:, -5:]], axis=1)
df_new = df_new.merge(df_tmp, on="ISO3_Code", how="inner")
columns = df.columns[-5:].values
names_dict = {}
for i, column in enumerate(columns):
    names_dict[column] = "%s_%s" % (FEATURE, column[:4])
df_new.rename(columns=names_dict, inplace=True)
df_new

,ISO3_Code,Country_Name,M49_Code,Infant_Mortality_Rate_2016,Infant_Mortality_Rate_2017,Infant_Mortality_Rate_2018,Infant_Mortality_Rate_2019,Infant_Mortality_Rate_2020
0,AFG,Afghanistan,4.0,67.5,64.8,62.3,60.1,58
1,ALB,Albania,8.0,9.4,9.4,9.5,9.7,9.8
2,DZA,Algeria,12.0,24.8,24.3,23.8,23.3,22.7
3,ASM,American Samoa,16.0,..,..,..,..,..
4,AND,Andorra,20.0,3.2,3,2.8,2.7,2.5
...,...,...,...,...,...,...,...,...
210,VIR,United States Virgin Islands,850.0,..,..,..,..,..
211,PSE,Palestine,275.0,18.6,18.1,17.5,17,16.5
212,YEM,Yemen,887.0,60.3,60.2,61.6,61.5,59.6
213,ZMB,Zambia,894.0,65.3,64.6,65.4,64.1,61.4


In [9]:
# add a column for average over the years
columns = df_new.columns[-5:]
df_cols = df_new[columns].applymap(lambda x: float(x) if x not in ["", None, ".."] else None)
df_new = pd.concat([df_new.iloc[:, :3], df_cols], axis=1)
df_new["%s_Avg" % FEATURE] = df_new[columns].mean(axis=1)
df_new.head()

,ISO3_Code,Country_Name,M49_Code,Infant_Mortality_Rate_2016,Infant_Mortality_Rate_2017,Infant_Mortality_Rate_2018,Infant_Mortality_Rate_2019,Infant_Mortality_Rate_2020,Infant_Mortality_Rate_Avg
0,AFG,Afghanistan,4.0,67.5,64.8,62.3,60.1,58.0,62.54
1,ALB,Albania,8.0,9.4,9.4,9.5,9.7,9.8,9.56
2,DZA,Algeria,12.0,24.8,24.3,23.8,23.3,22.7,23.78
3,ASM,American Samoa,16.0,NaN,NaN,NaN,NaN,NaN,NaN
4,AND,Andorra,20.0,3.2,3.0,2.8,2.7,2.5,2.84


In [10]:
# prettify the float
#df_new = df_new[df_new.columns[-6:]].applymap(na_action='ignore', func=lambda x: ".6f" % x)
with open("../../datasets/Processed/%s_Processed.csv" % FEATURE, "w", encoding="utf8") as f:
    df_new.to_csv(f, lineterminator="\n", index=False)

## Stats

In [11]:
df_new.isnull().sum()

ISO3_Code                      0
Country_Name                   0
M49_Code                       0
Infant_Mortality_Rate_2016    22
Infant_Mortality_Rate_2017    22
Infant_Mortality_Rate_2018    22
Infant_Mortality_Rate_2019    22
Infant_Mortality_Rate_2020    22
Infant_Mortality_Rate_Avg     22
dtype: int64